# Matrix Inversion as a QUBO problem
This notebook is a reference implementation for the paper "Floating-Point Calculations on a Quantum Annealer: Division and Matrix Inversion" by M. Rogers and R. Singleton, 2020 [1]. The paper describes an approach to floating-point calculations on a quantum annealer. Specifically, they introduce a binary representation of the floating point variables. This representation is discrete, as they demonstrate their technique for 4 bit and 8 bit accuracy. With this measure they explain and derive the inversion of a matrix as a QUBO problem, which is suitable to run on a DWave quantum annealer. 

[1] Rogers, Michael L., and Robert L. Singleton Jr. "Floating-point calculations on a quantum annealer: Division and matrix inversion." Frontiers in Physics 8 (2020): 265.

In [1]:
import numpy as np
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.embedding.chain_strength import scaled
import dwave.inspector

The main challenge in formulating problems for quantum annealers is the description as a binary problem. The authors formulate their problem as a QUBO problem. 

Scalar notation of a QUBO problem: 

\begin{equation} E_{qubo}(a_i, b_{i,j}; q_i) = \sum_{i} a_i q_i + \sum_{i < j} b_{i, j} q_i q_j 
\end{equation} 

Matrix (NxN) inversion is defined as: 

\begin{equation} M \cdot x = y \rightarrow x=M^{-1} \cdot y 
\end{equation} 

Formulate matrix inversion as a quadratic minimization problem with its minimum being the matrix inverse:

\begin{equation} H(x) = (Mx-y)^2 = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2 \sum_{ij=1}^{N} y_j M_{ji} x^i + \| y \|^2
\end{equation} 


We can obtain a floating point representation of each component of x by expanding in powers of 2 multiplied by boolean-valued variables $q_r^i \in \{0,1\}$ : 


\begin{equation} \chi^i = \sum_{r=0}^{R-1} 2^{-r} q_{r}^{i}
\end{equation} 

\begin{equation} x^i = 2\chi^i -1
\end{equation} 

In [2]:
def compute_chi(q, binary_range):
    # q: Value of binary qubits e.g. q = [0, 1, 0, 1]
    # Binary range is R: amount of binary variables to represent floating precision e.g. 4
    chi = 0
    for r in range(0, binary_range):
        chi += 2**(-r) * q[r]
    return chi

def compute_x(chi):
    # This is just an intermediate step...
    return 2*chi - 1

Let's confirm this with the smallest (-1.0) and largest element (2.75) in the range:

In [3]:
# Smallest element
q_start = np.array([0, 0, 0, 0])
# Largest element
q_end = np.array([1, 1, 1, 1])
# Range is length of bidary number
binary_range = q_start.shape[0]
# Compute value...
x_start = compute_x(compute_chi(q_start, binary_range))
x_end = compute_x(compute_chi(q_end, binary_range))
print(x_start)
print(x_end)

-1.0
2.75


So now our problem can be formulated by expressing x as a function $q_r^i$:

\begin{equation} H[q] = \sum_{i=1}^{N} \sum_{r=0}^{R-1} a_r^i q_r^i + \sum_{i=1}^{N} \sum_{i \neq j}^{N} \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_r^i q_s^i
\end{equation} 

With this we can formulate our QUBO coefficients. For a detailed derivation please refer to the paper:

\begin{equation} a_r^i = 4 \cdot 2^{-r} \sum_{k} M_{ki} \{ 2^{-r} M_{ki} - (y_k + \sum_{j} M_{kj}) \}
\end{equation} 

\begin{equation} b_{rs}^{ij} = 4 \cdot 2^{-(r+s)} \sum_{k} M_{ki} M_{kj} 
\end{equation} 

In the DWave system QUBO coefficients are addressed with a 1-D index, therefore we have to address the components with a 1-dimensional linear index. So the 2D-indices i (0, N-1) and r (0, R-1) are linarized with the index l. Where l is a usual row-major linear mapping index.

\begin{equation} l(i, r) = i \cdot R + r
\end{equation} 

\begin{equation} M_l = M_{i, r}
\end{equation} 

\begin{equation} i_l = [l / R]
\end{equation} 

\begin{equation} r_l = l  \%  R
\end{equation} 

\begin{equation} a_l = 4 \cdot 2^{-r_l} \sum_{k} M_{ki_l} \{ 2^{-r_l} M_{ki_l} - (y_k + \sum_{j} M_{kj}) \}
\end{equation} 



In [4]:
def compute_a_l(l, binary_range, M, y, alpha=20):
    # Implementation correct
    # l is the 1D-index
    # binary_range is the amount of digits to represent a floating number
    # M is a matrix of (N x N) 
    # y is a vector
    n = M.shape[0]
    a_l = 0
    # Un-map 1D index
    i_l = l // binary_range
    r_l = l % binary_range
    # Compute ...
    for k in range(n):
        temp_sum = np.sum(M, axis=1)
        a_l += M[k, i_l] * (2**(-r_l) * M[k, i_l] - (y[k] + temp_sum[k]))
    a_l = 4 * 2**(-r_l) * a_l
    return a_l * alpha

\begin{equation} b_{lm} = 4 \cdot 2^{-(r_l+r_{l'})} \sum_{k} M_{ki_l} M_{ki_m} 
\end{equation} 

In [5]:
def compute_b_l_m(l, m, binary_range, M, alpha=20):
    # Implementation correct
    # r is the position of the digit on the binary number
    # M is a matrix of (N x N) 
    # y is a vector
    n = M.shape[0]
    b_l = 0
    # Un-map 1D index
    i_l = l // binary_range
    i_m = m // binary_range
    r_l = l % binary_range
    r_m = m % binary_range
    # Compute ...
    for k in range(n):
        b_l += M[k, i_l] * M[k, i_m]
    b_l = 4 * (2**(-(r_l+r_m))) * b_l
    return b_l * alpha

Compute solution x from sampleset qubits

In [6]:
def compute_x_from_q(q_solution, r, n, i):
    # Implementation should be fine
    # Compute the coefficients of x from solved q: binary -> floating
    x = 0
    for j in range(n*r):
        i_l = j // r
        r_l = j % r
        if i == i_l :
            x += 2**(-r_l) * q_solution[j]
    x = 2 * x - 1
    return x

## Let's imagine a matrix

In [16]:
# Initialize matrix
# Test 1 (a):
M = np.array([[1/2, 3/2], [3/2, 1/2]])
#M = np.array([[2, -1], [-1/2, 1/2]])
y = np.array([1, 0])
# Initialize solution
#y = np.array([1, 0])
# y_norm = np.inner(y, y)
# M = 2*(M/y_norm) -1
# y = 2*(y/y_norm) -1
# max_v = max(np.max(M), np.max(y))
# min_v = min(np.min(M), np.min(y))
# M = 2 * ((M-min_v)/(max_v-min_v)) - 1
# y = 2 * ((y-min_v)/(max_v-min_v)) - 1
print(M)
print(y)
# Matrix size
n = M.shape[0]
# Binary bit size
r = 4
# 1-D range
total_l = n*r

[[0.5 1.5]
 [1.5 0.5]]
[1 0]


In [17]:
def normalize_matrix_equation(M,Y):
    # Compute norms of M, Y and X.
    mnorm = np.linalg.norm(M,-2) # "negative 2"-norm of M
    ynorm = np.linalg.norm(Y)    # ordinary 2-norm of Y
    xnorm = ynorm/mnorm
    # Normalize M and Y.
    m = M/mnorm
    y = Y/ynorm
    return m, y, xnorm

In [18]:
M, y, x_norm = normalize_matrix_equation(M, y)

## Construct QUBO problem as BQM for a DWave solver
To implement a usable problem for the DWave solver, we need to construct a matrix Q for our QUBO model. Q contains it's linear coefficient along the diagonal and the quadratic coefficients 

In [19]:
def construct_qubo_dict(M, y, binary_range):
    l = M.shape[0] * binary_range
    Q = {}
    for i in range(l):
        Q[(i, i)] = compute_a_l(i, binary_range, M, y)#+compute_b_l_m(i, i, binary_range, M)
        for j in range(i+1, l):
            Q[(i, j)] = compute_b_l_m(i, j, binary_range, M)
    return Q

In [20]:
Q = construct_qubo_dict(M, y, binary_range)
print(Q)

{(0, 0): -160.0, (0, 1): 100.0, (0, 2): 50.0, (0, 3): 25.0, (0, 4): 120.0, (0, 5): 60.0, (0, 6): 30.0, (0, 7): 15.0, (1, 1): -130.0, (1, 2): 25.0, (1, 3): 12.5, (1, 4): 60.0, (1, 5): 30.0, (1, 6): 15.0, (1, 7): 7.5, (2, 2): -77.5, (2, 3): 6.25, (2, 4): 30.0, (2, 5): 15.0, (2, 6): 7.5, (2, 7): 3.75, (3, 3): -41.875, (3, 4): 15.0, (3, 5): 7.5, (3, 6): 3.75, (3, 7): 1.875, (4, 4): -240.0, (4, 5): 100.0, (4, 6): 50.0, (4, 7): 25.0, (5, 5): -170.0, (5, 6): 25.0, (5, 7): 12.5, (6, 6): -97.5, (6, 7): 6.25, (7, 7): -51.875}


In [21]:
bqm = dimod.BQM.from_qubo(Q)
sampler = DWaveSampler()
chain_strength = scaled(bqm)
sampleset = EmbeddingComposite(sampler).sample(bqm, num_reads=1000, label="QUBO Matrix Inversion", chain_strength=chain_strength)
dwave.inspector.show(sampleset)
print(sampleset)

     0  1  2  3  4  5  6  7   energy num_oc. chain_.
0    0  1  1  1  0  1  1  1 -389.375      29     0.0
1    0  1  1  0  0  1  1  1 -379.375      21     0.0
2    0  1  1  1  0  1  1  0 -369.375       9     0.0
3    0  1  1  1  1  0  1  1 -369.375       9     0.0
4    0  0  1  1  1  1  1  1 -369.375      10     0.0
5    0  1  0  1  0  1  1  1 -369.375      13     0.0
6    0  1  1  0  1  0  1  1 -366.875      12     0.0
7    0  0  1  1  1  1  1  0 -366.875       7     0.0
8    0  0  1  1  1  1  0  1 -364.375      15     0.0
9    0  1  0  1  1  0  1  1 -364.375      10     0.0
10   0  0  1  0  1  1  1  1 -361.875       8     0.0
11   0  1  0  1  1  1  0  1 -361.875       9     0.0
12   0  1  1  1  1  0  1  0 -361.875      21     0.0
13   0  1  1  0  1  1  0  1 -360.625      13     0.0
14   0  1  0  1  1  1  1  0 -360.625       9     0.0
15   0  0  1  1  1  0  1  1 -359.375      16     0.0
16   0  1  1  1  1  1  0  1 -359.375       8     0.0
17   0  1  0  1  1  1  1  1 -359.375       6  

In [23]:
first_solution = np.array(list(sampleset.first.sample.items()))[:, 1]
print(first_solution)

[0 1 1 1 0 1 1 1]


In [24]:
x = np.zeros((n))
for i in range(n):
    x[i] = compute_x_from_q(np.array([0, 1, 1, 0, 0, 1, 1, 1]), r, n, i)
    # This gives -1: Implementation correct
    # x[i] = compute_x_from_q(np.ones((n*r)), r, n, i)
    # This gives 3: Implementation correct
print(x)

[0.5  0.75]


In [25]:
x = np.zeros((n))
# Feed through each component of x
for i in range(n):
    x[i] = compute_x_from_q(first_solution, r, n, i)
print(x)

[0.75 0.75]


In [26]:
classical_result = np.linalg.inv(M).dot(y)
print(classical_result)

[-0.25  0.75]


# Run matrix inversion on quantum annealer without intermediate steps

This method computes the solution of the linear equations using the annealing method, described above, without any intermediate steps. Just pass on the matrix, vector and binary range.

In [16]:
def matrix_inversion_dwave(M, y, binary_range=4):
    y_norm = np.inner(y,y)
    M = 2*M/y_norm -1
    y = 2*y/y_norm -1
    Q = construct_qubo_dict(M, y, binary_range)
    # Sampling solutions for 1000 reads
    offset = np.inner(y, y)
    # minimum = min(np.min(M), np.min(y))
    # maximum = max(np.max(M), np.max(y))
    # M = 2 * ((M-minimum)/(maximum-minimum)) - 1
    # y = 2 * ((y-minimum)/(maximum-minimum)) - 1
    # offset = 0
    #print("Offset: ", offset)
    #bqm = dimod.BinaryQuadraticModel(L, Q, offset, dimod.Vartype.BINARY)
    bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset=offset)
    sampleset = EmbeddingComposite(DWaveSampler()).sample(bqm, num_reads=1000, label="QUBO Matrix Inversion")
    first_solution = np.array(list(sampleset.first.sample.items()))[:, 1]
    x = np.zeros((n))
    # Feed through each component of x
    for i in range(n):
        x[i] = compute_x_from_q(first_solution, binary_range, n, i)
    return x #, sampleset

# 2 x 2 test cases from the paper

In [17]:
# Initialize matrix
# Test 1 (a):
M_a = np.array([[1/2, 3/2], [3/2, 1/2]])
y_a = np.array([1, 0])
# Test 1 (b):
M_b = np.array([[1/2, 3/2], [3/2, 1/2]])
y_b = np.array([0, 1])
# Test 1 (c):
M_c = np.array([[2, -1], [-1/2, 1/2]])
y_c = np.array([1, 0])
# Test 1 (d):
M_d = np.array([[1, 2], [1/2, 1/2]])
y_d = np.array([1, 0])
# Test 1 (e):
M_e = np.array([[3, 2], [2, 1]])
y_e = np.array([1, 1])
# Test 1 (f):
M_f = np.array([[3, 2], [2, 1]])
y_f = np.array([1, 1])
# Test 1 (g):
M_g = np.array([[0, -2], [-2, -3/3]])
y_g = np.array([1, 1/4])
# Test 1 (h):
M_h = np.array([[0, -2], [-2, -3/3]])
y_h = np.array([-0.5, -0.875])
# Test 1 (i): Ill-conditioned
M = np.array([[1, 2], [2, 3.999]])
y = np.array([4.0, 7.999])
# Test 1 (i): Pre-conditioned version of test 1 (i)
M_i = np.array([[1.80026, 1.6019], [1.6019, 4.19974]])
y_i = np.array([5.2007, 7.40013])
# Matrix size
n = M.shape[0]
# Binary bit size
r = 4
# 1-D range
total_l = n*r
Ms = [M_a, M_b, M_c, M_d, M_e, M_f, M_g, M_h, M_i]
ys = [y_a, y_b, y_c, y_d, y_e, y_f, y_g, y_h, y_i]

# 3 x 3 test cases from the paper

In [18]:
# # # Initialize matrix
# # Test 1 (a):
# M_a = np.array([[0.0, -2.0, 0.0], [-2.0, 1.5, 0.0], [0.0, 0.0, 1.0]])
# y_a = np.array([1.0, 0.25, 1.0])
# # # Test 1 (b):
# M_b = np.array([[0.0, -2.0, 0.0], [-2.0, 1.5, 0.0], [0.0, 0.0, 1.0]])
# y_b = np.array([1.0, 0.25, 0.0])
# # # Test 1 (c):
# M_c = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, -2.0], [0.0, -2.0, -1.5]])
# y_c = np.array([1.0, 0.0, 0.25])
# # # Test 1 (d):
# M_d = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, -2.0], [0.0, -2.0, -1.5]])
# y_d = np.array([1.0, 1.0, 0.25])
# # # Test 1 (e):
# M_e = np.array([[0.0, -2.0, 0.0], [-2.0, 1.5, 0.0], [0.0, 0.0, 1.0]])
# y_e = np.array([0.0, 1.0, 0.25])
# # # Test 1 (f):
# M_f = np.array([[-4.0, 6.0, 1.0], [8.0, -11.0, -2.0], [-3.0, 4.0, 1.0]])
# y_f = np.array([0.75, -1.25, 0.25])
# # # Test 1 (g):
# M_g = np.array([[6.1795, 11.8207, 2.0583], [15.673, -7.56717, -3.8520], [-5.6457, 7.96872, 15.9418]])
# y_g = np.array([1.4114, 0.9972, 9.9643])

# # Matrix size
# n = M.shape[0]
# # Binary bit size
# r = 4
# # 1-D range
# total_l = n*r

# Ms = [M_a, M_b, M_c, M_d, M_e, M_f, M_g, M_h, M_i]
# ys = [y_a, y_b, y_c, y_d, y_e, y_f, y_g, y_h, y_i]

Compute for on test case

In [19]:
result = matrix_inversion_dwave(M_b, y_b, binary_range=4)
print(result)

[ 0.75 -0.25]


Compute for all test cases

In [20]:
for i in range(len(Ms)):
    annealing_result = matrix_inversion_dwave(Ms[i], ys[i], binary_range=4)
    classical_result = np.linalg.inv(Ms[i]).dot(ys[i])
    print("Annealing: ", annealing_result, "Classical: ", classical_result)
    

Annealing:  [-0.25  0.75] Classical:  [-0.25  0.75]
Annealing:  [ 0.75 -0.25] Classical:  [ 0.75 -0.25]
Annealing:  [-0.25 -1.  ] Classical:  [1. 1.]
Annealing:  [2.75 0.75] Classical:  [-1.  1.]
Annealing:  [0.75 0.75] Classical:  [ 1. -1.]
Annealing:  [0.75 0.75] Classical:  [ 1. -1.]
Annealing:  [0.75 0.75] Classical:  [ 0.125 -0.5  ]
Annealing:  [0.75 0.75] Classical:  [0.3125 0.25  ]
Annealing:  [0.75 2.75] Classical:  [1.9996474  0.99932254]


| Annealing result | Classical result | 
| --- | --- | 
| [-0.25 0.75] | [-0.25 0.75] | 
| [ 0.75 -0.25] | [ 0.75 -0.25] |
| [-0.25 -1.  ] |  [1. 1.] |
|  [0.75 0.75] |  [-1.  1.] |
|  [0.75 0.75] |  [ 1. -1.] |
|  [0.75 0.75] |  [ 1. -1.] |
|  [-0.25 -0.25] |  [ 0.125 -0.5  ] |
|  [0.75 0.75] |  [0.3125 0.25  ] |
|  [2.75 0.75] |  [1.9996474  0.99932254] |